# GUI - Nine Men Morris
In diesem Abschnitt soll die graphische Oberfläche des Spiels entwickelt werden. Dies beinhaltet die Interaktion des Spielers mit dem Spielaufbau.

In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

## Mögliche Positionen
Die Grafiken unten zeigen zwei unterschiedliche Notationen für die Position der Spielsteine. Rechts ist die offizielle Notation des Welt-Mühle-Dachverbands (WMD) abgebildet, während links die interne Notation zu sehen ist.

### WMD-Notation
"Die Spalten werden von links beginnend von a bis g benannt. Die Zeilen werden von unten beginnend von 1 bis 7 durchnummeriert. Gültige Feldnamen sind beispielsweise a1, d2, e3 oder g7.

Bei dieser Notation gehört nicht zu jeder Buchstaben- und Zahlenkombination aus a..g und 1..7 auch ein Feld. Ungültige Feldbezeichnungen sind beispielsweise b3, d4, f5 oder g6. Diese Notation ähnelt der beim Schach, wo die 64 Felder von a..h und 1..8 benannt werden. Allerdings gehört beim Schach zu jeder Zahlen/Buchstabenkombination auch ein Feld.

Beim Setzen schreibt man das Zielfeld auf, zum Beispiel a1 oder e4. Beim Ziehen werden die Namen des durch Minus getrennten Quell- und Zielfeldes verwendet, zum Beispiel f2-f4 oder d3-d2. Wird ein Mühle gebildet und ein Stein geschlagen, dann wird der geschlagene Stein durch x getrennt und an den Zug angefügt, beispielweise f2-f4xd5 (Zug von f2 nach f4, Mühle und geschlagen wird d5)." - Dr. Rainer Rosenberger, http://muehlespieler.de/download/muehle_lehrbuch.pdf

### Interne Notation
Intern werden die Positionen als Tupel bestehend aus Ringnummer und Zellnummer beschrieben. Die Ringe, in der Abbildung farbig markiert, sind dabei von außen beginnend mit 0, 1 und 2 nummeriert. Für die Bezeichnung der Zellen wird in jedem Ring oben links mit 0 begonnen und im Uhrzeigersinn bis 7 weitergezählt. So ergibt sich eine eindeutige Notation.

<img src="board_positions.png" alt="Mögliche Positionen" width="800"/>

## Definitionen
Zunächst werden einige Definitionen für die gesamte Implementation festgesetzt: 
- ring = eines der 3 Quadrate (Wertebereich: 0-2)
- cell = ein Punkt auf einem Ring (Wertebereich: 0-7)
- position = Tupel: (ring, cell)
- board = Liste aus 3 Listen, die angeben, ob, und wenn ja welcher Stein dort sitzt. Dabei symbolisiert jede Liste einen Ring. In jeder Liste sind neun Werte. Je einer für eine Zelle im Ring. Eine 0 sagt dabei aus, dass an der entsprechenden Position kein Stein ist. Eine 1 oder 2 stehen für einen Stein des zugehörigen Spielers.
- remaining = \[Anzahl noch nicht gesetzter Steine Spieler 1, Anzahl noch nicht gesetzte Steine Spieler 2\]
- state = \[remaining, board\]

## Importe
Zur Visualisierung des Spielbrettes wird [ipycanvas](https://ipycanvas.readthedocs.io/en/latest/index.html) verwendet.

In [ ]:
%run ./Muehle_Logic.ipynb
import ipycanvas
from ipycanvas import MultiCanvas

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Konstanten

Für das Aussehen des Spielfelds:
* BOARD_SIZE = Größe des Spielfeldes in Pixeln
* DOT_RADIUS = Radius der schwarzen kleinen Punkte, die mögliche Positionen markieren (in Abhängigkeit von der Spielfeldgröße)
* PIECE_RADIUS = Radius der Spielsteine
* COLOUR = Farben der [dots, pieces_player_1, pieces_player_2]
* COLOUR_HINT = Farbe, in der mögliche Zielfelder markiert werden
* COLOUR_OPPONENT = Farbe, in der entfernbare Steine markiert werden
* PADDING = relativer Abstand des äußersten Quadrats zum Spielfeldrand
* DISTANCE = relativer Abstand zwischen den Quadraten des Spielfelds
* TRANSPARENCY_DEFAULT = Standardwert für die Transparenz
* TRANSPARENCY_HINT = Transparenz der Kreise möglicher Zielfelder und entfernbarer Steine

Für die Funktion des Spielfelds:
* START_STATE = Startzustand des Spielfelds, beide Spieler haben noch keinen Stein gesetzt und das Spielfeld ist leer

In [ ]:
# Aussehen
BOARD_SIZE      = 400
DOT_RADIUS      = BOARD_SIZE*0.025
PIECE_RADIUS    = BOARD_SIZE*0.04
COLOUR          = ['black', 'white', 'sienna']
COLOUR_HINT     = 'green'
COLOUR_OPPONENT = 'red'
PADDING         = 0.05
DISTANCE        = 0.15
TRANSPARENCY_DEFAULT = 1.0
TRANSPARENCY_HINT    = 0.5

# Spiel
START_STATE = [[9,9],[          # Anzahl zu setzender Steine (Spieler_1 (Weiß), Spieler_2 (Braun))
            [0,0,0,0,0,0,0,0], # ring 0
            [0,0,0,0,0,0,0,0], # ring 1
            [0,0,0,0,0,0,0,0]  # ring 2
      ]]

 `row()` liefert einen relativen Wert für die Position der eingegebenen Reihe. `col()` macht dies für die Spalten.

In [ ]:
#Reihe von oben nach unten (0-6)
def row(number):
    return PADDING + DISTANCE * number
#Spalte von links nach rechts (0-6)
def col(number):
    return row(number)

`POSITIONS` beschreibt die möglichen Punkte, an denen Steine sitzen können.  `wmd_dic` überführt die Positionen in die Schreibweise des WMD.

In [ ]:
POSITIONS = [([col(0), row(0)], [col(3), row(0)], [col(6), row(0)], [col(6), row(3)], [col(6), row(6)], [col(3), row(6)], [col(0), row(6)], [col(0), row(3)]), #ring 0
             ([col(1), row(1)], [col(3), row(1)], [col(5), row(1)], [col(5), row(3)], [col(5), row(5)], [col(3), row(5)], [col(1), row(5)], [col(1), row(3)]), #ring 1
             ([col(2), row(2)], [col(3), row(2)], [col(4), row(2)], [col(4), row(3)], [col(4), row(4)], [col(3), row(4)], [col(2), row(4)], [col(2), row(3)])] #ring 2

wmd_dic = {
    "a1":POSITIONS[0][6],
    "a4":POSITIONS[0][7],
    "a7":POSITIONS[0][0],
    "b2":POSITIONS[1][6],
    "b4":POSITIONS[1][7],
    "b6":POSITIONS[1][0],
    "c3":POSITIONS[2][6],
    "c4":POSITIONS[2][7],
    "c5":POSITIONS[2][0],
    "d1":POSITIONS[0][5],
    "d2":POSITIONS[1][5],
    "d3":POSITIONS[2][5],
    "d5":POSITIONS[2][1],
    "d6":POSITIONS[1][1],
    "d7":POSITIONS[0][1],
    "e3":POSITIONS[2][4],
    "e4":POSITIONS[2][3],
    "e5":POSITIONS[2][2],
    "f2":POSITIONS[1][4],
    "f4":POSITIONS[1][3],
    "f6":POSITIONS[1][2],
    "g1":POSITIONS[0][4],
    "g4":POSITIONS[0][3],
    "g7":POSITIONS[0][2]
}

## Zeichenfunktionen
`init_canvas()` erstellt den Aufbau des Spielfeldes. Dies erfolgt mit Hilfe von ipycanvas. Das Spielbrett (`board`) besteht aus drei übereinanderliegenden Ebenen. Ebene 0 ist der Hintergrund, ein beiges Quadrat. Ebene 1 beinhaltet schwarzen die Striche und Punkte auf dem Spielfeld. Diese setzen sich aus drei schwarzen Quadraten, viel horizontalen bzw. waagrechten Linien und 24 Punkten, die die möglichen Positionen der Steine markieren, zusammen. Dabei wird bei der Erstellung mit relativen Positionsangaben gearbeitet, um das Spielfeld einfach skalieren zu können. In der Ebene 2 liegen schließlich die Spielsteine. 

In [ ]:
def init_canvas():
    #board[Hintergrund, Linien, Steine]
    board = MultiCanvas(3, width = BOARD_SIZE, height = BOARD_SIZE)

    # Hintergrund
    board[0].fill_style = '#ffffcc'
    board[0].fill_rect(0, 0, BOARD_SIZE)

    # Strichstärke
    board[1].line_width = 5

    # Quadrate
    board[1].stroke_rect(BOARD_SIZE*col(0), BOARD_SIZE*row(0), BOARD_SIZE*(1-row(0)-col(0))) #ring 0
    board[1].stroke_rect(BOARD_SIZE*col(1), BOARD_SIZE*row(1), BOARD_SIZE*(1-row(1)-col(1))) #ring 1
    board[1].stroke_rect(BOARD_SIZE*col(2), BOARD_SIZE*row(2), BOARD_SIZE*(1-row(2)-col(2))) #ring 2

    # Mittelinien
    board[1].begin_path()
    board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(0)) #oben
    board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(2))
    board[1].move_to(BOARD_SIZE*col(6), BOARD_SIZE*row(3)) #rechts
    board[1].line_to(BOARD_SIZE*col(4), BOARD_SIZE*row(3))
    board[1].move_to(BOARD_SIZE*col(3), BOARD_SIZE*row(6)) #unten
    board[1].line_to(BOARD_SIZE*col(3), BOARD_SIZE*row(4))
    board[1].move_to(BOARD_SIZE*col(0), BOARD_SIZE*row(3)) #links
    board[1].line_to(BOARD_SIZE*col(2), BOARD_SIZE*row(3))
    board[1].stroke()

    # Punkte (außen, mitte, innen)
    for ring in POSITIONS:
        for x,y in ring:
            board[1].fill_arc(BOARD_SIZE*x, BOARD_SIZE*y, DOT_RADIUS, 0, 360)
    return board

In [ ]:
board = init_canvas()

Die Funktion `draw_piece()` zeichnet an einer zu übergebenden Position einen Spielstein für den eingegebenen Spieler. Dieser besteht aus einem mit der Spielerfarbe gefüllten Kreis als Hintergrund und zwei Innenringen um die Steine realistischer aussehen zu lassen.

In [ ]:
def draw_piece(ring, cell, player):
    #Hintergrund
    board[2].fill_style = COLOUR[player]
    board[2].fill_arc(BOARD_SIZE*POSITIONS[ring][cell][0], BOARD_SIZE*POSITIONS[ring][cell][1], PIECE_RADIUS, 0, 360)
    #Ringe
    if player == 1:
        board[2].stroke_style = 'silver' 
    else:
        board[2].stroke_style = 'chocolate'
    board[2].stroke_arc(BOARD_SIZE*POSITIONS[ring][cell][0], BOARD_SIZE*POSITIONS[ring][cell][1], PIECE_RADIUS,     0, 360)
    board[2].stroke_arc(BOARD_SIZE*POSITIONS[ring][cell][0], BOARD_SIZE*POSITIONS[ring][cell][1], PIECE_RADIUS*0.7, 0, 360)
    board[2].stroke_arc(BOARD_SIZE*POSITIONS[ring][cell][0], BOARD_SIZE*POSITIONS[ring][cell][1], PIECE_RADIUS*0.3, 0, 360)

Mit Hilfe von `highlight_positions()` werden alle Positionen aus einer Liste mit einem transparenten Kreis markiert. Falls keine andere Farbe übergeben wird, wird die in den Konstanten definierte default-Farbe verwendet.

In [ ]:
def highlight_positions(highlight_positions, colour = COLOUR_HINT):
    with ipycanvas.hold_canvas(board):
        for position in highlight_positions:
            ring, cell = position
            board[2].fill_style = colour
            board[2].global_alpha = TRANSPARENCY_HINT
            board[2].fill_arc(BOARD_SIZE*POSITIONS[ring][cell][0], BOARD_SIZE*POSITIONS[ring][cell][1], PIECE_RADIUS, 0, 360)

`update_board()` zeichnet die obere Ebene des Spielbretts nach jeder Veränderung neu. So werden Positionsänderungen oder Hinweise zu möglichen Zielfeldern und schlagbaren Steinen sichtbar. Zusätzlich wird hier die Aktualisierung der Statusanzeige angestoßen.
Durch den optionalen Parameter message ist es möglich die Statusausgabe zu überschreiben.

In [ ]:
def update_board(game, message=''):
    one_below_other = widgets.VBox([turn_label, status_label, pieces_status_label, stones_player_one, stones_player_two, restart_button])
    side_by_side = widgets.HBox([board, one_below_other])
    display(side_by_side)

    with ipycanvas.hold_canvas(board):
        board[2].clear()
        board[2].global_alpha = TRANSPARENCY_DEFAULT
        for ring in range(3):
            for cell in range(8):
                player = game.state[1][ring][cell]
                if player in [1, 2]: draw_piece(ring, cell, player)
        update_status_widgets(message)    

## Die Klasse Game - Status des Spielfelds
Der Status speichert die aktuelle Situation des Spielfeldes. Die Klasse beinhaltet mehrere Variablen. `state` gibt wie in der Definition beschrieben an, ob und wenn ja welcher Stein sich an einer Position auf dem Spielfeld befindet. Dabei gilt:
- 0 = kein Stein
- 1 = weißer Stein
- 2 = brauner Stein

Außerdem beinhaltet die Variable Angaben über die Steine, die die beiden Spieler nocht setzen können.
`current_player` gibt an, welcher Spieler, 1 oder 2, gerade am Zug ist. `mills` ist die Menge aller aktuell bestehenden Mühlen. Die Einträge werden in einem Tupel der Form (Spieler, Ring, Zelle) gespeichert. In `number_stones_to_remove` wird gespeichert, wie viele Steine der aktuelle Spieler aufgrund von Mühlen noch entfernen darf. `selected_stone` ist ein Tupel, das die Position eines ausgewählten Steins speichert. Dieses Tupel hat die Form `(ring, cell)`. Dies ist notwendig für Zug- und Springvorgänge. Ist die Variable ungleich `None`, befindet sich der Spieler gerade in einer Zug- oder Sprungaktion. Ist die Variable  belegt, so hat der Spieler bereits den zu versetzenden Stein ausgewählt und muss noch auf das Zielfeld klicken. Die Position ist in Phase 2 des Spiels relevant, um valide Zielfelder zu ermitteln. In der Phase 1 kann es vorkommen, dass kein Stein gesetzt wird, weil kein passendes Feld augewählt wurde, um zu verhindern, dass der Spieler trotzdem wechselt, wird die Variable `do_not_change` auf True gesetzt und diese vor Spielerwechsel abgefragt. Das Gleiche geschieht in den Phasen 2 und 3, falls kein passender Stein ausgewählt wird. `game_over`ist False, solange das Spiel noch läuft.

Die `__init__()` Funktion setzt die Werte der Game-Klasse. Dabei kann falls notwendig der aktuelle Spieler übergeben werden, falls beispielsweise Spieler 2 starten soll. Standardmäßig wird als Zustand de
r Startzustand übergeben, wobei davon zuvor eine Deepcopy angefertigt werden muss. Dies ist notwendig, da ansonsten die Refernz auf `START_STATE` übergeben wird und die Konstante somit im Laufe des Spiel verändert werden würde.

Mit Hilfe der Funktion `__str__()` kann der Status später mit dem Befehl `print(Game)` formatiert ausgegeben werden. Dies ist besonders zum Überprüfen des aktuellen Spielstandes hilfreich.

In [ ]:
class Game():
    def __init__(self, current_player = 1):
        self.state = copy.deepcopy(START_STATE)
        self.current_player = current_player
        self.mills = set()
        self.number_stones_to_remove = 0
        self.selected_stone = (None, None)
        self.do_not_change = False
        self.game_over = False
    
    def __str__(self):
        return "state: " + str(self.state) \
            + "\ncurrent player: " + str(self.current_player) \
            + "\nmills: " + str(self.mills) \
            + "\nnumber stones to remove: " + str(self.number_stones_to_remove) \
            + "\nselected stone: " + str(self.selected_stone) \
            + "\ndo not change: " + str(self.do_not_change) \
            + "\ngame over: " + str(self.game_over)

Die Funktion `play_game()` steuert das Spiel. Sie wird später aufgerufen, wenn der Spieler irgendwo hin klickt. Wenn das Spiel noch nicht vorbei ist, wird überprüft, ob der Klick an einer Positionen, also im Bereich um einen schwarzen Punkt, erfolgt ist. Wenn ja, kann eine der folgenden Situationen eintreten, wobei die erste aufgelistete Situation Priorität hat:
- **Es müssen noch Steine entfernt werden**  - Aufgrund von Mühlen die zuvor geschlossen wurden, dürfen noch Steine entfernt werden. Der zuvor geklickte Stein wird der Funktion `remove_stone()` übergeben, die ihn vom Spielfeld löscht.
- **Der Spieler befindet sich in Phase 1** - Auf die ausgewählte Position wird mit Hilfe von `place_stone()` ein Stein gesetzt.
- **Es wurde zuvor kein Stein ausgewählt** - Da sich der Spieler wie zuvor überprüft nicht in Phase 1 und somit in der Zug- oder Springphase befindet, und bisher kein Startfeld ausgewählt wurde, kann dies nun geschehen. Dazu wird die Funktion `select_stone()` aufgerufen.
- **Es wurde ein Stein ausgewählt** - Da sich der Spieler in der Zug- oder Springphase befindet, handelt es sich bei dem ausgewählten Feld um ein Zielfeld, auf dass der zuvor ausgewählte Stein mittels `move_stone()` bewegt wird.

Falls nach abarbeiten der Situationen noch Steine zu entfernen sind, werden diese markiert. Außerdem wird überprüft, ob das Spiel zu Ende ist. Falls sich der Spieler nicht in einem Zwischenzustand von Phase 2 oder 3 (Startfeld ausgewählt, Zielfeld noch nicht bekannt) befindet, keine Steine mehr entfernt werden müssen, das Spiel nicht zu Ende ist und vorher alle korrekt gelaufen ist, ist der nächste Spieler an der Reihe und das Spielfeld wird aktualisiert.

In [ ]:
def play_game(self, x, y):
    if self.game_over: return

    phase = get_player_phase(self.state, self.current_player)
    stone = self.get_clicked_stone(x, y)

    # Situationen überprüfen
    if stone == None:
        return
    if self.number_stones_to_remove > 0:
        self.remove_stone(stone)
    elif phase == 1:
        self.place_stone(stone)
    elif self.selected_stone != (None, None): # die ausgewählte Position ist ein Zielfeld
        self.move_stone(stone)
    elif phase == 2 or phase == 3: # der Stein soll zum Verschieben oder Springen ausgewählt werden
        self.select_stone(stone)
    else:
        change_status_label('Du befindest Dich anscheinend in keiner gültigen Spielphase. Irgendwas ist schief gegangen... Sorry!')

    if self.number_stones_to_remove > 0:
        update_board(self)
        highlight_positions(get_opponent_beatable_stones(self.state, self.current_player), colour = COLOUR_OPPONENT)

    if finished(self.state):
        self.game_over = True

    if self.selected_stone == (None, None) and not self.number_stones_to_remove > 0 and not self.game_over and not self.do_not_change:
        self.current_player = opponent(self.current_player) # Spielerwechsel, wenn nicht noch eine Aktion ausgeführt werden muss
        update_board(self)
Game.play_game = play_game
del play_game

Die Funktion `get_clicked_stone()` überprüft, ob die übergebenen x- und y- Koordinaten auf einer validen Position, also einem Spielfeldpunkt liegen. Wenn ja, wird diese Position in der Form `(ring, cell)` zurückgegeben, ansonsten `None`. Zur Ermittlung wird über alle möglichen Positionen iteriert und geprüft, ob die Klick-Koordinaten in einem Pufferbereich rund um den Mittelpunkt der Position liegen. Ist dies für beide Koordinaten der Fall, wurde der angeklickte Punkt gefunden.

In [ ]:
def get_clicked_stone(self, x, y):
    stone = None
    #TODO: Schleife abbrechen, sobal stone gesetzt wurde
    for ring in POSITIONS:
        for pos in ring:
            check_x = pos[0] * BOARD_SIZE - PIECE_RADIUS/2 < x < pos[0] * BOARD_SIZE + PIECE_RADIUS/2
            check_y = pos[1] * BOARD_SIZE - PIECE_RADIUS/2 < y < pos[1] * BOARD_SIZE + PIECE_RADIUS/2
            if check_x and check_y:
                stone = (POSITIONS.index(ring), ring.index(pos))
    return stone
Game.get_clicked_stone = get_clicked_stone
del get_clicked_stone

In Phase 1 wird durch `place_stone()` überprüft, ob das angeklickte Feld noch frei ist. Wenn ja, wird dort ein Stein des Spielers positioniert und die eigenen Steinanzahl in `remaining` um 1 reduziert. Anschließend muss noch überprüft werde, ob durch die Aktion eine oder sogar mehrere Mühlen entstanden ist. War die Position nicht gültig, wird eine Fehlermeldung geworfen und der Spieler erhält einen neuen Versuch. 

In [ ]:
def place_stone(self, stone):
    if stone in empty_positions(self.state):
        self.do_not_change = False
        ring, cell = stone
        self.state[1][ring][cell] = self.current_player
        self.state[0][self.current_player-1] = self.state[0][self.current_player-1]-1 
        self.number_stones_to_remove = handle_mills(self)
        update_board(self)
    else:
        self.do_not_change = True
        change_status_label('Du kannst Deinen Stein nur auf ein leeres Feld setzen. Bitte probiere es erneut!')
Game.place_stone = place_stone
del place_stone

`remove_stone()` entfernt den übergebenen Stein vom Spielfeld, falls es erlaubt ist, diesen zu entfernen. Dazu  werden alle schlagbaren Steine des Gegners ermittelt und überprüft, ob der angeklickte Stein dazu gehört. Ist dies der Fall, wird der Stein enfernt, indem an dieser Stelle eine `0` eingetragen wird. Andernfalls wird eine Fehlermeldung geworfen und der Spieler kann einen anderen Stein zum Entfernen auswählen. Falls noch mehr Steine entfernt werden müssen, werden die möglichen Positionen am Ende markiert.

In [ ]:
def remove_stone(self, stone):
    selected_ring, selected_cell = stone
    beatable_stones = get_opponent_beatable_stones(self.state, self.current_player)
    if (selected_ring, selected_cell) in beatable_stones:
        self.state[1][selected_ring][selected_cell] = 0
        self.number_stones_to_remove = self.number_stones_to_remove - 1
        update_board(self)
    else:
        change_status_label('Du kannst diesen Stein nicht entfernen. Bitte probiere es erneut!')
    if self.number_stones_to_remove > 0:
        highlight_positions(get_opponent_beatable_stones(self.state, self.current_player), colour = COLOUR_OPPONENT)
Game.remove_stone = remove_stone
del remove_stone

`move_stone()` bewegt den zuvor ausgewählten Stein, falls zulässig, auf die übergebene Position. Das Startfeld ist dabei in der Variable `selected_stone` gespeichert. Damit das Zielfeld valide ist, muss es in der Liste sein, die durch `next_positions()` ausgehend vom ausgewählten Stein ermittelt werden. Ist dies der Fall, wird das alte Feld auf `0` gesetzt und das Zielfeld mit dem Wert des aktuellen Spielers belegt. Anschließend muss überprüft werden, ob neue Mühlen entstanden sind

In [ ]:
def move_stone(self, stone):
    old_ring, old_cell = self.selected_stone
    new_ring, new_cell = stone
    possible_positions = next_positions(self.state, self.current_player, old_ring, old_cell)
    if stone in possible_positions:
        self.state[1][new_ring][new_cell] = self.current_player
        self.state[1][old_ring][old_cell] = 0
        self.number_stones_to_remove = handle_mills(self)
        self.selected_stone = (None, None)
        update_board(self)
    else:
        change_status_label('Du kannst Deinen Stein nicht auf dieses Feld setzen. Bitte probiere es erneut!')
Game.move_stone = move_stone
del move_stone

Falls sich der Spieler in der Zug- oder Springphase befindet, und noch kein Startfeld ausgewählt wurde, wird `select_stone()` aufgerufen. Diese Funktion überprüft, ob der übergebene Stein ein eigener Stein ist. Wenn ja wird er in der Variable `selected_stone` gespeichert und ausgehend davon mögliche Zielfelder berechnet, um diese hervorheben zu können. Nach erfolgreicher Durchführung befindet sich der Spieler in einem Zwischenzustand, da das Zielfeld noch nicht bekannt ist.

In [ ]:
def select_stone(self, stone):
    if stone in player_pieces(self.state, self.current_player): # prüfen,ob der ausgewählte Stein dem Spieler gehört
        self.do_not_change = False
        self.selected_stone = stone
        change_status_label('Zielort für den eigenen Stein auswählen.')
        ring, cell = stone
        possible_positions = next_positions(self.state, self.current_player, ring, cell) # ausgehend vom ausgewählten Stein, erlaubte Zielfelder berechnen
        highlight_positions(possible_positions) # erlaubte Zielfelder anzeigen
    else:
        self.do_not_change = True
        change_status_label('Das ist nicht Dein Stein. Bitte probiere es erneut!')
Game.select_stone = select_stone
del select_stone

## Die Statusanzeige - Initialisierung und Funktionen der Status-Widgets und des Restart-Button

Die Methode `on_button_clicked(b)` wird durch den restart_button aufgerufen und setzt den Status des Spieles wieder auf den Startzustand.

In [ ]:
def on_button_clicked(b):
    global game
    game = Game()
    update_board(game)

An dieser Stelle findet die Initialisierung der unterschiedlichen Statuselemente statt.
-  Das turn_label ist ein Label, das den Spieler anzeigt, der aktuell an der Reihe ist
-  Das status_label ist für weitere Statusanzeigen vorgesehen, wie die Aufforderung an den aktuellen Spieler, im Fall einer Mühle, einen Stein des Gegners zu entfernen.
-  Das pieces_status_label zeigt je nach Spielphase entweder die Anzahl der Steine, die noch gesetzt werden können (Phase 1), oder die noch übrig sind (Phase 2/3). 
-  Der restart_button ruft die Methode `on_button_clicked(b)` auf und setzt somit das Spiel auf den Startzustand zurück
-  stones_player_one und stones_player_two zeigen jeweils einen horizontalen Balken, der in Phase 1 nach jedem gesetzten Stein kleiner wird, bis keiner mehr vorhanden ist. In Phase 2 und 3 ist die Auslenkung des Balkens von der Anzahl der Steine des jeweiligen Spielers abhängig. Somit ist schnell erkennbar, welcher Spieler gerade vorne liegt.


In [ ]:
turn_label = widgets.Label(value='Spieler x ist an der Reihe')

status_label = widgets.Label(value='aktueller Status:')

pieces_status_label = widgets.Label(value='Setzbare Steine:')

restart_button = widgets.Button(description='restart')
restart_button.on_click(on_button_clicked)

stones_player_one = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 1',
                bar_style= 'info', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

stones_player_two = widgets.FloatProgress(
                value=9,
                max=9,
                min=0,
                description='Spieler 2',
                bar_style='warning', # 'success', 'info', 'warning', 'danger' or ''
                orientaiton='horizontal'
            )

Die Methode `change_status_label(text)` nimmt einen beliebigen Status-Text entgegen und setzt diesen mit dem Präfix 'Status: ' als Text des status_label.

In [ ]:
def change_status_label(text):
    status_label.value = 'Status: '+text

Die Methode `update_status_widgets()` aktualisiert die unterschiedlichen Statusanzeigen passend zur aktuellen Spielphase.

In [ ]:
def update_status_widgets(message):
    player_color = 'Weiß'
    if(game.current_player == 2):
        player_color = 'Braun'
    turn_label.value='Spieler '+ str(game.current_player)+ ' (' + player_color + ') an der Reihe.'
    if(get_player_phase(game.state, game.current_player) == 1):
        change_status_label('Bitte einen Stein auf dem Spielfeld platzieren.')
    if(get_player_phase(game.state, game.current_player) == 2):
        change_status_label('Bitte einen Stein zum Ziehen auswählen.')
    if(get_player_phase(game.state, game.current_player) == 3):
        change_status_label('Bitte einen Stein zum Springen auswählen.')
    if(game.state[0][0] == game.state[0][1] == 0):
        pieces_status_label.value = 'Verbleibende Steine:'
        stones_player_one.value = len(player_pieces(game.state, 1))
        stones_player_two.value = len(player_pieces(game.state, 2))
    else:
        pieces_status_label.value = 'Setzbare Steine:'
        stones_player_one.value = game.state[0][0]
        stones_player_two.value = game.state[0][1]
    if(message != ''):
        change_status_label(message)
    if(utility(game.state, game.current_player) == 1):
        change_status_label('Spieler '+ str(game.current_player)+ ' hat gewonnen!')
    if(game.number_stones_to_remove > 0):
        if(game.number_stones_to_remove == 1):
            change_status_label('Mühle! Bitte noch 1 Stein des Gegners entfernen.') 
        else:
            change_status_label('Mühle! Bitte noch ' + str(game.number_stones_to_remove) + ' Steine des Gegners entfernen.') 

## Spielen

Schließlich kann das Spielfeld erzeugt und ein Spiel gestartet werden. Dabei wird bei jedem Mausklick auf das Spielfeld `play_game()` mit den Koordinaten des Klicks aufgerufen.

In [ ]:
game = Game()
board[2].on_mouse_down(game.play_game)
update_board(game)